In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import regularizers
import pickle

2023-03-15 15:06:03.502859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
aggregated_data = pickle.load( open( '../data/processed/20230304/aggregated_data', 'rb' ) )

[train_mps_raw_pca, valid_mps_raw_pca, test_mps_raw_pca,
    train_mps_bg_pca, valid_mps_bg_pca, test_mps_bg_pca,
    train_mps_fg_pca, valid_mps_fg_pca, test_mps_fg_pca,
    train_indices_raw_pca, valid_indices_raw_pca, test_indices_raw_pca,
    train_indices_bg_pca, valid_indices_bg_pca, test_indices_bg_pca,
    train_indices_fg_pca, valid_indices_fg_pca, test_indices_fg_pca,
    train_embedding_raw_pca, valid_embedding_raw_pca, test_embedding_raw_pca,
    train_embedding_bg_pca, valid_embedding_bg_pca, test_embedding_bg_pca,
    train_embedding_fg_pca, valid_embedding_fg_pca, test_embedding_fg_pca,
    train_vgg_raw_pca, valid_vgg_raw_pca, test_vgg_raw_pca,
    train_vgg_bg_pca, valid_vgg_bg_pca, test_vgg_bg_pca,
    train_vgg_fg_pca, valid_vgg_fg_pca, test_vgg_fg_pca,
    train_panns_clip_raw_pca, valid_panns_clip_raw_pca, test_panns_clip_raw_pca,
    train_panns_clip_bg_pca, valid_panns_clip_bg_pca, test_panns_clip_bg_pca,
    train_panns_clip_fg_pca, valid_panns_clip_fg_pca, test_panns_clip_fg_pca,
    train_panns_embedding_raw_pca, valid_panns_embedding_raw_pca, test_panns_embedding_raw_pca,
    train_panns_embedding_bg_pca, valid_panns_embedding_bg_pca, test_panns_embedding_bg_pca,
    train_panns_embedding_fg_pca, valid_panns_embedding_fg_pca, test_panns_embedding_fg_pca,
    y_train, y_valid, y_test] = aggregated_data

In [3]:
# # YAMNet
# train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_embedding_raw_pca, train_vgg_raw_pca), axis = 1)
# valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_embedding_raw_pca, valid_vgg_raw_pca), axis = 1)
# test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_embedding_raw_pca, test_vgg_raw_pca), axis = 1)

# train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_embedding_bg_pca, train_vgg_bg_pca), axis = 1)
# valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_embedding_bg_pca, valid_vgg_bg_pca), axis = 1)
# test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_embedding_bg_pca, test_vgg_bg_pca), axis = 1)

# train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_embedding_fg_pca, train_vgg_fg_pca), axis = 1)
# valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_embedding_fg_pca, valid_vgg_fg_pca), axis = 1)
# test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_embedding_fg_pca, test_vgg_fg_pca), axis = 1)

# train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
# valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
# test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [4]:
# # VGG
# train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_vgg_raw_pca), axis = 1)
# valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_vgg_raw_pca), axis = 1)
# test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_vgg_raw_pca), axis = 1)

# train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_vgg_bg_pca), axis = 1)
# valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_vgg_bg_pca), axis = 1)
# test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_vgg_bg_pca), axis = 1)

# train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_vgg_fg_pca), axis = 1)
# valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_vgg_fg_pca), axis = 1)
# test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_vgg_fg_pca), axis = 1)

# train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
# valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
# test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

In [5]:
# # including panns embeddings
# train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_panns_embedding_raw_pca), axis = 1)
# valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_panns_embedding_raw_pca), axis = 1)
# test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_panns_embedding_raw_pca), axis = 1)

# train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_panns_embedding_bg_pca), axis = 1)
# valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_panns_embedding_bg_pca), axis = 1)
# test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_panns_embedding_bg_pca), axis = 1)

# train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_panns_embedding_fg_pca), axis = 1)
# valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_panns_embedding_fg_pca), axis = 1)
# test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_panns_embedding_fg_pca), axis = 1)

# train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
# valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
# test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

# Lazy predict

In [6]:
# from lazypredict.Supervised import LazyClassifier
# clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(train_vgg_bg_pca, valid_vgg_bg_pca, y_train, y_valid)
# models

# Tensorflow parameter tuning

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [8]:
def build_model_narrow(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_raw.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=200, step=10),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

def build_model_wide(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_all.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.2, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=10, max_value=500, step=50),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [9]:
def build_model_AUC_narrow(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_raw.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.0, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=16, max_value=256, step=16),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#         loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.AUC(name='AUC')],
    )

    return model

def build_model_AUC_wide(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_all.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.0, max_value=0.5, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=16, max_value=512, step=16),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.AUC(name='AUC')],
    )

    return model

In [10]:
def build_model_bACC_narrow(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_raw.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.0, max_value=0.5, sampling="linear")
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=16, max_value=256, step=16),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    threshold = hp.Float("threshold", min_value=0.3, max_value=0.7, sampling="linear")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy(name='binary_accuracy', threshold=threshold)],
    )

    return model

def build_model_bACC_wide(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(train_data_all.shape[1]), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.0, max_value=0.5, sampling="linear")
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=16, max_value=512, step=16),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
        model.add(layers.Dropout(rate=drop_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    threshold = hp.Float("threshold", min_value=0.3, max_value=0.7, sampling="linear")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy(name='binary_accuracy', threshold=threshold)],
    )

    return model

In [11]:
# panns

directory = '../model/BinaryCrossentropy_bACC'
feat_name = '0315_pannsEmbedding'
objective = 'val_binary_accuracy'


# including panns embeddings
train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_panns_embedding_raw_pca), axis = 1)
valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_panns_embedding_raw_pca), axis = 1)
test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_panns_embedding_raw_pca), axis = 1)

train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_panns_embedding_bg_pca), axis = 1)
valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_panns_embedding_bg_pca), axis = 1)
test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_panns_embedding_bg_pca), axis = 1)

train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_panns_embedding_fg_pca), axis = 1)
valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_panns_embedding_fg_pca), axis = 1)
test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_panns_embedding_fg_pca), axis = 1)

train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)


tuner_raw = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_raw_'+feat_name)

tuner_bg = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_bg_'+feat_name)

tuner_fg = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_fg_'+feat_name)

tuner_all = kt.BayesianOptimization(build_model_bACC_wide,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_all_'+feat_name)


tuner_all.search(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_bg.search(train_data_bg, y_train, epochs=50, validation_data=(valid_data_bg, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_raw.search(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_fg.search(train_data_fg, y_train, epochs=50, validation_data=(valid_data_fg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_bACC/kt_Bayesian_raw_0315_pannsEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_bACC/kt_Bayesian_bg_0315_pannsEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_bACC/kt_Bayesian_fg_0315_pannsEmbedding/tuner0.json
INFO:tensorflow:Reloading Tuner from ../model/BinaryCrossentropy_bACC/kt_Bayesian_all_0315_pannsEmbedding/tuner0.json
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Oracle triggered exit


In [12]:
# YAMNet

directory = '../model/BinaryCrossentropy_bACC'
feat_name = '0315_YAMNetEmbedding'
objective = 'val_binary_accuracy'


# YAMNet
train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_embedding_raw_pca, train_vgg_raw_pca), axis = 1)
valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_embedding_raw_pca, valid_vgg_raw_pca), axis = 1)
test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_embedding_raw_pca, test_vgg_raw_pca), axis = 1)

train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_embedding_bg_pca, train_vgg_bg_pca), axis = 1)
valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_embedding_bg_pca, valid_vgg_bg_pca), axis = 1)
test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_embedding_bg_pca, test_vgg_bg_pca), axis = 1)

train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_embedding_fg_pca, train_vgg_fg_pca), axis = 1)
valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_embedding_fg_pca, valid_vgg_fg_pca), axis = 1)
test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_embedding_fg_pca, test_vgg_fg_pca), axis = 1)

train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)


tuner_raw = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_raw_'+feat_name)

tuner_bg = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_bg_'+feat_name)

tuner_fg = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_fg_'+feat_name)

tuner_all = kt.BayesianOptimization(build_model_bACC_wide,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_all_'+feat_name)

tuner_all.search(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_bg.search(train_data_bg, y_train, epochs=50, validation_data=(valid_data_bg, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_raw.search(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_fg.search(train_data_fg, y_train, epochs=50, validation_data=(valid_data_fg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 100 Complete [00h 00m 08s]
val_binary_accuracy: 0.6586183905601501

Best val_binary_accuracy So Far: 0.6753856539726257
Total elapsed time: 00h 25m 32s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# VGG

directory = '../model/BinaryCrossentropy_bACC'
feat_name = '0315_vggEmbedding'
objective = 'val_binary_accuracy'


# VGG
train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_vgg_raw_pca), axis = 1)
valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_vgg_raw_pca), axis = 1)
test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_vgg_raw_pca), axis = 1)

train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_vgg_bg_pca), axis = 1)
valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_vgg_bg_pca), axis = 1)
test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_vgg_bg_pca), axis = 1)

train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_vgg_fg_pca), axis = 1)
valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_vgg_fg_pca), axis = 1)
test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_vgg_fg_pca), axis = 1)

train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)


tuner_raw = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_raw_'+feat_name)

tuner_bg = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_bg_'+feat_name)

tuner_fg = kt.BayesianOptimization(build_model_bACC_narrow,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_fg_'+feat_name)

tuner_all = kt.BayesianOptimization(build_model_bACC_wide,
                     objective=objective,
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory=directory,
                     project_name='kt_Bayesian_all_'+feat_name)

tuner_all.search(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_bg.search(train_data_bg, y_train, epochs=50, validation_data=(valid_data_bg, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_raw.search(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)
tuner_fg.search(train_data_fg, y_train, epochs=50, validation_data=(valid_data_fg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 4 Complete [00h 00m 16s]
val_binary_accuracy: 0.6398390531539917

Best val_binary_accuracy So Far: 0.7149564027786255
Total elapsed time: 00h 00m 47s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
0.48296           |0.27406           |dr
1                 |3                 |num_layers
160               |16                |units_0
relu              |relu              |activation
0.00041268        |0.0013825         |lr
0.49779           |0.51157           |threshold
240               |144               |units_1
128               |96                |units_2
128               |176               |units_3

Epoch 1/50
373/373 [==============================] - 1s 2ms/step - loss: 0.8158 - binary_accuracy: 0.5667 - val_loss: 0.6022 - val_binary_accuracy: 0.6794
Epoch 2/50
373/373 [==============================] - 1s 1ms/step - loss: 0.7100 - binary_accuracy: 0.6133 - val_loss: 0.5859 - val_binary_accuracy: 0.6875
Epoch 3/50
373/373 [===================

In [ ]:
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# tuner_all.search(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
# tuner_bg.search(train_data_bg, y_train, epochs=50, validation_data=(valid_data_bg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
# tuner_raw.search(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
# tuner_fg.search(train_data_fg, y_train, epochs=50, validation_data=(valid_data_fg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

# Load models

In [ ]:
# all
tuner_YAMNet_all = kt.BayesianOptimization(build_model_AUC_wide,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_all_0314_YAMNetEmbedding')

tuner_VGG_all = kt.BayesianOptimization(build_model_AUC_wide,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_all_0314_vggEmbedding')

tuner_panns_all = kt.BayesianOptimization(build_model_AUC_wide,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_all_0314_pannsEmbedding')

# raw
tuner_YAMNet_raw = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_raw_0314_YAMNetEmbedding')

tuner_VGG_raw = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_raw_0314_vggEmbedding')

tuner_panns_raw = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_raw_0314_pannsEmbedding')


# bg
tuner_YAMNet_bg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_bg_0314_YAMNetEmbedding')

tuner_VGG_bg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_bg_0314_vggEmbedding')

tuner_panns_bg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_bg_0314_pannsEmbedding')


# fg
tuner_YAMNet_fg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_fg_0314_YAMNetEmbedding')

tuner_VGG_fg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_fg_0314_vggEmbedding')

tuner_panns_fg = kt.BayesianOptimization(build_model_AUC_narrow,
                     objective='AUC',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
                     directory='../model/BinaryCrossentropy_valAUC',
                     project_name='kt_Bayesian_fg_0314_pannsEmbedding')

In [ ]:
tuner_YAMNet_all.results_summary(1)
print('--------------------------------------------------')
tuner_VGG_all.results_summary(1)
print('--------------------------------------------------')
tuner_panns_all.results_summary(1)

In [ ]:
tuner_YAMNet_raw.results_summary(1)
print('--------------------------------------------------')
tuner_VGG_raw.results_summary(1)
print('--------------------------------------------------')
tuner_panns_raw.results_summary(1)

In [ ]:
tuner_YAMNet_bg.results_summary(1)
print('--------------------------------------------------')
tuner_VGG_bg.results_summary(1)
print('--------------------------------------------------')
tuner_panns_bg.results_summary(1)

In [ ]:
tuner_YAMNet_fg.results_summary(1)
print('--------------------------------------------------')
tuner_VGG_fg.results_summary(1)
print('--------------------------------------------------')
tuner_panns_fg.results_summary(1)

# Use the best parameters to build a model

In [ ]:
best_hps=tuner_panns_raw.get_best_hyperparameters(num_trials=1)[0]
model = tuner_panns_raw.hypermodel.build(best_hps)
history = model.fit(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
val_AUC_per_epoch = history.history['val_AUC']
best_epoch = val_AUC_per_epoch.index(max(val_AUC_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner_panns_raw.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_data_raw, y_train, epochs=best_epoch, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
eval_result = hypermodel.evaluate(test_data_raw, y_test)
print("[test loss, test accuracy]:", eval_result)

# Plot AUC

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix

In [ ]:
y_pred = hypermodel.predict(test_data_raw).ravel() 
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

# plot the ROC curve
plt.plot(fpr, tpr, label=f'AUC = {auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
(tpr+1-fpr)/2